Syllable - Joseph Jojoe

In [15]:
# Imports dependencies
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa
from tensorflow.keras import regularizers
from keras.models import Model
import pathlib
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import string
import random
from keras.preprocessing.text import Tokenizer

In [16]:
# Loads in syllable data
dataframe = pd.read_csv("../input/nlpsyllable/preprocessed.txt",
                        sep=",",
                        encoding="ISO-8859-1",
                        names=["word", "label"])
# Necessary to specify str type for pandas columns
dataframe = dataframe.astype(str)
words = dataframe['word'].tolist()
labels = dataframe['label'].tolist()
# Converts each label to numpy array
for i in range(0, len(labels)):
    labels[i] = list(labels[i])
    for j in range(0, len(labels[i])):
        labels[i][j] = int(labels[i][j])
for i in range(0, len(labels)):
    labels[i] = np.array(labels[i])

In [17]:
# Vectorises syllable strings by treating each character as a token
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts(words)
words = tokenizer.texts_to_sequences(words)
for i in range(0, len(words)):
    words[i] = np.array(words[i], dtype=float)

padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(
    words, padding="post", maxlen=15
)
padded_outputs = tf.keras.preprocessing.sequence.pad_sequences(
    labels, padding="post", maxlen=15
)

# Normalisation
maximum_token = 64
for element in range(0, len(words)):
    words[element] = words[element] / maximum_token

In [18]:
# shuffles data
seed = random.random()
random.seed(seed)
random.shuffle(padded_inputs)
random.seed(seed)
random.shuffle(padded_outputs)

# splits into training and validation sets (80-20 split)
validation_inputs = padded_inputs[-35497:]
validation_outputs = padded_outputs[-35497:]
padded_inputs = padded_inputs[:-35497]
padded_outputs = padded_outputs[:-35497]

In [19]:
# Custom loss function - mean of binary crossentropy and mean squared error
def mean_weighted_bce_mse(y_true, y_prediction):
    y_true = tf.cast(y_true, tf.float32)
    y_prediction = tf.cast(y_prediction, tf.float32)

    # Binary crossentropy with weighting
    epsilon = 1e-6
    positive_weight = 4.108897148948174
    loss_positive = y_true * tf.math.log(y_prediction + epsilon)
    loss_negative = (1 - y_true) * tf.math.log(1 - y_prediction + epsilon)
    bce_loss = tf.math.reduce_mean(tf.math.negative(positive_weight * loss_positive + loss_negative))
    
    # Mean squared error
    mse = tf.keras.losses.MeanSquaredError()
    mse_loss = mse(y_true, y_prediction)

    averaged_bce_mse = (bce_loss + mse_loss) / 2
    return averaged_bce_mse

In [20]:
def inception_module(inputs):
    inception_branch_1 = tf.keras.layers.Conv1D(128, kernel_size=1, strides=2, activation="tanh", activity_regularizer=tf.keras.regularizers.l2(1e-5))(inputs)
    inception_branch_1 = tf.keras.layers.ZeroPadding1D(padding=(0, 15 - inception_branch_1.shape[1]))(inception_branch_1)

    inception_branch_2 = tf.keras.layers.Conv1D(128, kernel_size=1, activation="tanh", activity_regularizer=tf.keras.regularizers.l2(1e-5))(inputs)
    inception_branch_2 = tf.keras.layers.Conv1D(128, kernel_size=3, strides=2, activation="tanh", activity_regularizer=tf.keras.regularizers.l2(1e-5))(inception_branch_2)
    inception_branch_2 = tf.keras.layers.ZeroPadding1D(padding=(0, 15 - inception_branch_2.shape[1]))(inception_branch_2)

    inception_branch_3 = tf.keras.layers.AveragePooling1D(pool_size=3, strides=2)(inputs)
    inception_branch_3 = tf.keras.layers.Conv1D(128, kernel_size=3, activation="tanh", activity_regularizer=tf.keras.regularizers.l2(1e-5))(inception_branch_3)
    inception_branch_3 = tf.keras.layers.ZeroPadding1D(padding=(0, 15 - inception_branch_3.shape[1]))(inception_branch_3)

    inception_branch_4 = tf.keras.layers.Conv1D(128, kernel_size=1, activation="tanh", activity_regularizer=tf.keras.regularizers.l2(1e-5))(inputs)
    inception_branch_4 = tf.keras.layers.Conv1D(128, kernel_size=3, activation="tanh", activity_regularizer=tf.keras.regularizers.l2(1e-5))(inception_branch_4)
    inception_branch_4 = tf.keras.layers.Conv1D(128, kernel_size=3, strides=2, activation="tanh", activity_regularizer=tf.keras.regularizers.l2(1e-5))(inception_branch_4)
    inception_branch_4 = tf.keras.layers.ZeroPadding1D(padding=(0, 15 - inception_branch_4.shape[1]))(inception_branch_4)

    inception_output = tf.keras.layers.concatenate([inception_branch_1, inception_branch_2, inception_branch_3, inception_branch_4, inputs])
    return inception_output

In [21]:
inputs = tf.keras.Input(shape=(15,))
embedded_inputs = tf.keras.layers.Embedding(64, 256, mask_zero=True)(inputs)
embedded_inputs = tf.keras.layers.Dropout(0.25)(embedded_inputs)

x = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(256, return_sequences=True, dropout=0.1, activity_regularizer=tf.keras.regularizers.l2(1e-5)))(embedded_inputs)
x = tf.keras.layers.concatenate([x, embedded_inputs])
x = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(256, return_sequences=True, dropout=0.1, activity_regularizer=tf.keras.regularizers.l2(1e-5)))(x)
x = tf.keras.layers.concatenate([x, embedded_inputs])
x = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(256, return_sequences=True, activity_regularizer=tf.keras.regularizers.l2(1e-5)))(x)

inception_output = inception_module(embedded_inputs)
inception_output = tf.keras.layers.MaxPooling1D()(inception_output)
inception_output =  tf.keras.layers.ZeroPadding1D(padding=(0, 15 - inception_output.shape[1]))(inception_output)
inception_output = tf.keras.layers.Dropout(0.5)(inception_output)
inception_output = inception_module(inception_output)
inception_output = tf.keras.layers.MaxPooling1D()(inception_output)
inception_output =  tf.keras.layers.ZeroPadding1D(padding=(0, 15 - inception_output.shape[1]))(inception_output)

output = tf.keras.layers.concatenate([x, inception_output, embedded_inputs])
output = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(256, activation="relu"))(output)
output = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(64, activation="relu"))(output)
output = tf.keras.layers.GlobalMaxPool1D()(output)
output = tf.keras.layers.Dense(15, activation="sigmoid")(output)

metrics = ["binary_accuracy",
           tfa.metrics.F1Score(num_classes=15, threshold=0.5),
           tfa.metrics.HammingLoss(mode='multilabel', threshold=0.5),
           tf.keras.metrics.Recall(),
           tf.keras.metrics.Precision(),
           tf.keras.metrics.AUC(multi_label=True, num_labels=15)]

model = tf.keras.models.Model(inputs=inputs, outputs=output)
model.compile(optimizer="adam",
              loss=mean_weighted_bce_mse,
              metrics=metrics,
              steps_per_execution=128)

In [22]:
model.summary()

In [23]:
history = model.fit(padded_inputs,
                    padded_outputs,
                    validation_data=(validation_inputs, validation_outputs),
                    epochs=60,
                    batch_size=128)